In [ ]:
# Imports
import cv2
import os
import numpy as np

In [ ]:
# Resize the data
folders=['00000','01000','02000','03000']
for curFolder in folders:
    for curFile in os.listdir(curFolder):
        curImg=cv2.imread('{0}/{1}'.format(curFolder,curFile))
        curImg=cv2.resize(curImg,(128,128))
        cv2.imwrite('{0}/{1}'.format(curFolder,curFile),curImg)
    
# Data for Aakash
coloredFaces=[]
curFolder='00000'
for curFile in os.listdir(curFolder):
    if(curFile[0]=='_'):
        coloredFaces.append(curFile.replace('_',''))

In [ ]:
# Face Black and white 

# Data for training
trainX=[]
trainY=[]
folders=['00000']
for curFolder in folders:
    for curFile in os.listdir(curFolder):
        curImg=cv2.imread('{0}/{1}'.format(curFolder,curFile))
        trainX.append(curImg)
        if(curFile[0]=='_'):
            trainY.append(1)
        else:
            trainY.append(0)

trainX=np.array(trainX)
trainY=np.array(trainY)
            
# Model
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

torch.manual_seed(42)

class faceClass(nn.Module):
    def __init__(self):
        super(faceClass,self).__init__()
        self.conv1=nn.Conv2d(3,16,(3,3), stride=1, padding=0).cuda()
        self.conv2=nn.Conv2d(16,64,(3,3), stride=1, padding=0).cuda()
        self.max2=nn.MaxPool2d(2, stride=2).cuda()
        self.conv3=nn.Conv2d(64,256,(3,3), stride=1, padding=0).cuda()
        self.conv4=nn.Conv2d(256,512,(3,3), stride=1, padding=0).cuda()
        self.max4=nn.MaxPool2d(2, stride=2).cuda()
        self.conv5=nn.Conv2d(512,512,(3,3), stride=1, padding=0).cuda()
        self.max5=nn.MaxPool2d(2, stride=2).cuda()
        self.conv6=nn.Conv2d(512,3,(3,3), stride=1, padding=0).cuda()
        self.l1=nn.Linear(3*11*11,1).cuda()
        self.sigm=nn.Sigmoid().cuda()
        
    def forward(self,inputs):
        out=self.conv1(inputs).cuda()
        #out=self.max1(out).cuda()
        out=self.conv2(out).cuda()
        out=self.max2(out).cuda()
        out=self.conv3(out).cuda()
        #out=self.max3(out).cuda()
        out=self.conv4(out).cuda()
        out=self.max4(out).cuda()
        out=self.conv5(out).cuda()
        out=self.max5(out).cuda()
        out=self.conv6(out).cuda()
        #out=self.max6(out).cuda()
        #print("Size of out is {0}".format(out.size()))
        out=self.l1(out.view(-1,3*11*11))
        out=self.sigm(out)
        return out

model=faceClass().cuda()
criterion = torch.nn.BCELoss(weight=torch.FloatTensor([0.99]).cuda())
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
batchSize=16
epochs=10000
    
train=0
if(train==1):
    numBatches=int(trainX.shape[0]/batchSize)
    for curEpoch in range(epochs):
        totalLoss=0
        for curBatch in range(numBatches):
            model.zero_grad()
            X=Variable(torch.from_numpy(trainX[curBatch*batchSize:(curBatch+1)*batchSize].reshape(batchSize,3,128,128).astype(np.float32))).cuda()
            Y=Variable(torch.from_numpy(trainY[curBatch*batchSize:(curBatch+1)*batchSize].astype(np.float32))).cuda()
            output=model(X).cuda()
            curLoss=criterion(output,Y)
            totalLoss+=curLoss.item()
            curLoss.backward()
            optimizer.step()
        if(curEpoch % 5==0):
            print("{0} TotalLoss {1}".format(curEpoch,totalLoss))
        if(curEpoch %100==0):
            torch.save(model,"ColoredFaceDetectionModel")
print("Cell Execution Completed")

In [ ]:
# Creating the dataset for B&W Classification
# Create the trainX and trainY after manually marking the facial with an undescore at the beginning of the file name
import os
import numpy as np

model1=torch.load("ColoredFaceDetectionModel")
trainX=[]
trainY=[]
folders=['00000','01000','02000','03000']
for curFolder in folders:
    for curFile in os.listdir(curFolder):
        curImg=cv2.imread('{0}/{1}'.format(curFolder,curFile))
        trainX.append(curImg)
        modelVal=model1(Variable(torch.from_numpy(curImg.reshape(1,3,128,128).astype(np.float32))).cuda()).cpu().detach().numpy()[0][0]
        trainY.append(modelVal)

trainX=np.array(trainX)
trainY=[1 if x >0.2 else 0 for x in trainY]
trainY=np.array(trainY)

# Save the array
np.save("trainXGAN",trainX)
np.save("trainYGAN",trainY)

print("Cell Execution Completed")

In [ ]:
import numpy as np
trainX=np.load("trainXGAN.npy")
trainY=np.load("trainYGAN.npy")

In [ ]:
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch


class Generator(nn.Module):
    def __init__(self,latentDim,numClasses):
        super(Generator, self).__init__()
        self.latentDim=latentDim*latentDim
        self.numClasses=numClasses
        self.label_emb = nn.Embedding(2,self.latentDim).cuda()
        # The GAN Generator will be a CNN
        self.conv1=nn.Conv2d(1,4,(3,3), stride=1, padding=0).cuda()
        self.max1=nn.MaxPool2d(2, stride=2).cuda()
        self.conv2=nn.Conv2d(4,8,(3,3), stride=1, padding=0).cuda()
        self.max2=nn.MaxPool2d(2, stride=1).cuda()
        self.conv3=nn.Conv2d(8,32,(2,2), stride=1, padding=0).cuda()
        self.max3=nn.MaxPool2d(2, stride=1).cuda()
        self.conv4=nn.Conv2d(32,64,(2,2), stride=1, padding=0).cuda()
        self.max4=nn.MaxPool2d(2, stride=1).cuda()
        self.conv5=nn.Conv2d(64,64,(2,2), stride=1, padding=0).cuda()
        self.max5=nn.MaxPool2d(2, stride=1).cuda()
        self.conv6=nn.Conv2d(64,3,(2,2), stride=1, padding=0).cuda()
        self.max6=nn.MaxPool2d(2, stride=1).cuda()
        self.l1=nn.Linear(3*116*116,3*128*128).cuda()

    def forward(self, noise, labelVal):
        gen_input = torch.mul(noise,self.label_emb(labelVal)).view(-1,1,256,256).cuda()
        out=self.conv1(gen_input).cuda()
        out=self.max1(out).cuda()
        out=self.conv2(out).cuda()
        out=self.max2(out).cuda()
        out=self.conv3(out).cuda()
        out=self.max3(out).cuda()
        out=self.conv4(out).cuda()
        out=self.max4(out).cuda()
        out=self.conv5(out).cuda()
        out=self.max5(out).cuda()
        out=self.conv6(out).cuda()
        out=self.max6(out).cuda()
        out=self.l1(out).view(-1,3,128,128).cuda()
        print(out.size())
        return out

class Discriminator(nn.Module):
    def __init__(self,numClasses):
        super(Discriminator, self).__init__()
        self.numClasses=numClasses
        self.conv1=nn.Conv2d(1,4,(3,3), stride=1, padding=0).cuda()
        self.max1=nn.MaxPool2d(2, stride=2).cuda()
        self.conv2=nn.Conv2d(4,8,(3,3), stride=1, padding=0).cuda()
        self.max2=nn.MaxPool2d(2, stride=2).cuda()
        self.conv3=nn.Conv2d(8,16,(3,3), stride=1, padding=0).cuda()
        self.max3=nn.MaxPool2d(2, stride=2).cuda()
        self.conv4=nn.Conv2d(16,64,(3,3), stride=1, padding=0).cuda()
        self.max4=nn.MaxPool2d(2, stride=2).cuda()
        self.conv5=nn.Conv2d(64,64,(3,3), stride=1, padding=0).cuda()
        self.max5=nn.MaxPool2d(2, stride=2).cuda()
        self.adv_layer = nn.Sequential(nn.Linear(45, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(45, self.numClasses), nn.Softmax())

    def forward(self, discrim_input):
        out=self.conv1(discrim_input).cuda()
        out=self.max1(out).cuda()
        out=self.conv2(out).cuda()
        out=self.max2(out).cuda()
        out=self.conv3(out).cuda()
        out=self.max3(out).cuda()
        out=self.conv4(out).cuda()
        out=self.max4(out).cuda()
        out=self.conv5(out).cuda()
        out=self.max5(out).cuda()
        out=self.conv6(out).cuda()
        out=self.max6(out).cuda()
        print(out.size())
        validity=self.adv_layer(out).cuda()
        label=self.aux_layer(out).cuda()
        return validity,label
    
print("Cell Execution Completed")

import torch
import numpy as np

# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.BCELoss()

# Optimizers
beta1=0.1
beta2=0.01
latentDim=256
numClasses=2
epochs=1

# Initialize generator and discriminator

generator = Generator(latentDim,2)
discriminator = Discriminator(2)
learningRateG=0.00001
learningRateD=0.001
generatorExtraCount=8
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learningRateG)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learningRateD)

FloatTensor = torch.FloatTensor
LongTensor = torch.LongTensor  
batchSize=4
numBatches=1

for epoch in range(epochs):
    totalGLoss=0
    totalDLoss=0
    for curBatch in range(numBatches):
        #inpData=curData[choiceList,:].astype(np.float32)
        #labels=labelVals[choiceList].astype(np.int)
        real_imgs=Variable(torch.from_numpy(trainX[curBatch*batchSize:(curBatch+1)*batchSize].reshape(batchSize,3,128,128).astype(np.float32))).cuda()
        labels=Variable(torch.from_numpy(trainY[curBatch*batchSize:(curBatch+1)*batchSize])).cuda()
        # Adversarial ground truths
        valid = Variable(FloatTensor(batchSize, 1).fill_(1.0), requires_grad=False).cuda()
        fake = Variable(FloatTensor(batchSize, 1).fill_(0.0), requires_grad=False).cuda()

        # GENERATOR TRAINING
        # Set the gradients for the Generator
        optimizer_G.zero_grad()
        # Sample noise and random labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batchSize, latentDim*latentDim)))).cuda()
        gen_labels = Variable(LongTensor(np.random.randint(0, numClasses, batchSize))).cuda()
        # Generate a batch of data
        gen_imgs = generator(z, gen_labels)
        print("Size of gen_imgs is {0}".format(gen_imgs.size()))
        # Loss measures generator's ability to fool the discriminator
        validity,pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))
        g_loss.backward()
        optimizer_G.step()

        # DISCRIMINATOR TRAINING
        optimizer_D.zero_grad()
        #Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()
        # Calculate discriminator accuracy
        #pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        #gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        #d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        totalGLoss=totalGLoss + g_loss.item()
        totalDLoss=totalDLoss + d_loss.item()
    if(epoch %1==0):
        print("[Epoch {0}/{1}] [D loss:{2}] [G loss: {3}]".format(epoch, epochs, totalDLoss,totalGLoss))